In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import joblib  # Used for loading the trained model
import json 
from scipy.stats import moment
from skimage.feature import local_binary_pattern
from skimage.feature import graycomatrix as graycomatrix, graycoprops as graycoprops
from skimage import measure
import mahotas
import ast


In [2]:
# Function to normalize an image
def normalize_image(image):
    return cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)


In [3]:
# Function to resize an image
def resize_image(image, size):
    return cv2.resize(image, size)


In [4]:
# Function to extract color moments
def extract_color_moments(image):
    moments_r = moment(image[:, :, 0].ravel(), moment=[1, 2, 3])
    moments_g = moment(image[:, :, 1].ravel(), moment=[1, 2, 3])
    moments_b = moment(image[:, :, 2].ravel(), moment=[1, 2, 3])
    return moments_r, moments_g, moments_b


In [5]:
# Function to extract Haralick texture features
def extract_haralick_features(gray_image):
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    entropy = graycoprops(glcm, 'ASM')[0, 0]
    return contrast, correlation, entropy


In [6]:
# Function to extract LBP features
def extract_lbp_features(gray_image):
    lbp = local_binary_pattern(gray_image, 8, 1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    return hist


In [7]:
# Function to extract Zernike moments
def extract_zernike_moments(gray_image):
    return mahotas.features.zernike_moments(gray_image, radius=10, degree=8)


In [8]:
# Function to extract Hu moments
def extract_hu_moments(gray_image):
    return cv2.HuMoments(cv2.moments(gray_image)).flatten()


In [9]:
# Function to apply Gabor filters and extract features
def apply_gabor_filters(gray_image):
    gabor_features = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        kernel = cv2.getGaborKernel((21, 21), 5.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        fimg = cv2.filter2D(gray_image, cv2.CV_8UC3, kernel)
        gabor_features.append(fimg.mean())
        gabor_features.append(fimg.var())
    return gabor_features


In [10]:
# Function to extract contour-based features
def extract_contour_features(binary_image, gray_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea) if contours else None
    if largest_contour is None:
        return None
    area = cv2.contourArea(largest_contour)
    perimeter = cv2.arcLength(largest_contour, True)
    x, y, w, h = cv2.boundingRect(largest_contour)
    aspect_ratio = float(w) / h
    contour_image = np.zeros_like(gray_image)
    cv2.drawContours(contour_image, [largest_contour], -1, (255), thickness=cv2.FILLED)
    eccentricity = measure.regionprops(measure.label(contour_image))[0].eccentricity
    hull = cv2.convexHull(largest_contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    solidity = area / hull_area
    return area, perimeter, aspect_ratio, eccentricity, hull_area, hull_perimeter, solidity


In [11]:
# Function to process a single image and extract features
def process_single_image(image_path, input_size):
    image = cv2.imread(image_path)
    
    # Normalize and resize the image
    normalized_image = normalize_image(image)
    resized_image = resize_image(normalized_image, input_size)

    # Convert the resized image to grayscale
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    
    # Extract features
    moments_r, moments_g, moments_b = extract_color_moments(image)
    contrast, correlation, entropy = extract_haralick_features(gray_image)
    lbp_features = extract_lbp_features(gray_image)
    zernike_moments = extract_zernike_moments(gray_image)
    hu_moments = extract_hu_moments(gray_image)
    gabor_features = apply_gabor_filters(gray_image)

    # Extract HSV histograms
    hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv_image], [0], None, [256], [0, 256]).flatten()
    hist_s = cv2.calcHist([hsv_image], [1], None, [256], [0, 256]).flatten()
    hist_v = cv2.calcHist([hsv_image], [2], None, [256], [0, 256]).flatten()

    # Extract GLCM properties
    glcm = graycomatrix(gray_image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    glcm_props = [graycoprops(glcm, prop).ravel()[0] for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']]

    # Extract contour-based features
    binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    contour_features = extract_contour_features(binary_image, gray_image)
    
    if contour_features is not None:
        area, perimeter, aspect_ratio, eccentricity, hull_area, hull_perimeter, solidity = contour_features
        features = {
                    'Contrast': contrast, 'Dissimilarity': glcm_props[1], 'Homogeneity': glcm_props[2], 'Energy': glcm_props[3],
                    'Correlation': glcm_props[4],'Area': area, 'Perimeter': perimeter,'Aspect_Ratio': aspect_ratio, 
                    'Eccentricity': eccentricity, 'Hull_Area': hull_area, 'Hull_Perimeter': hull_perimeter, 'Solidity': solidity,
                    'Hue_Histogram': hist_h.tolist(), 'Saturation_Histogram': hist_s.tolist(), 'Value_Histogram': hist_v.tolist(),
                    'Moments_R': moments_r.tolist(), 'Moments_G': moments_g.tolist(), 'Moments_B': moments_b.tolist(),
                    'LBP_Features': lbp_features.tolist(),'Zernike_Moments': zernike_moments.tolist(), 'Hu_Moments': hu_moments.tolist(),
                    'Gabor_Features': gabor_features
                }
        
        return features
    
        return None


In [12]:
# Function to process list columns to match training data format
def process_list_column(features):
    processed_features = {}
    for col, values in features.items():
        if isinstance(values, list):
            for i, value in enumerate(values):
                processed_features[f"{col}_{i+1}"] = value
        else:
            processed_features[col] = values
    return processed_features



In [13]:
# Load the trained logistic regression model
model = joblib.load(r"C:\Users\havar\Downloads\Project\ML_model_output\DecisionTreeClassifier_model.pkl")

# Path to the single image to test
image_path = r"C:\Users\havar\Downloads\Project\Leaf Images Database\Terminalia arjuna\0002_0187.JPG"
input_size = (224, 224)  



In [14]:
features = process_single_image(image_path, input_size)
if features:
    processed_features = process_list_column(features)
    
    # Convert to DataFrame and match the order of columns used during training
    feature_df = pd.DataFrame([processed_features])
    training_columns = pd.read_csv(r"C:\Users\havar\Downloads\Project\handcrafted_features\extracted.csv")
   # Remove extra whitespaces
    training_columns.columns = training_columns.columns.str.strip()
    
    # Drop 'Class' and 'Filename' columns
    training_columns = training_columns.drop(['Class', 'Filename'], axis=1, errors='ignore')
    
    # Reindex feature_df based on training_columns
    feature_df = feature_df.reindex(columns=training_columns.columns, fill_value=0)
    # Remove feature names from the DataFrame
    feature_df.columns = range(feature_df.shape[1])
    
    # Make predictions
    prediction = model.predict(feature_df)
    print(f'Predicted Class: {prediction[0]}')
    
else:
    print('No valid contour found in the image.')


Predicted Class: Citrus aurantiifolia
